In [7]:
import numpy as np
import random
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from chatbot import bag_of_words, tokenize, stem
from chatbot import NeuralNet

[nltk_data] Downloading package punkt to /Users/tejas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
with open('intents.json', 'r') as f:
	intents = json.load(f)
all_words = []
tags = []
xy = []
# looping through each sentence in our intents patterns
for intent in intents['intents']:
	tag = intent['tag']
	tags.append(tag)
	for pattern in intent['patterns']:
		# tokenize each word in the sentence pattern
		# append to our list of words
		w = tokenize(pattern)
		all_words.extend(w)
		xy.append((w, tag))
ignore_words = set(['?', '.', '!'])
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
X_train = []
y_train = []
for (pattern_sentence,tag) in xy:
	bag = bag_of_words(pattern_sentence, all_words)
	#PyTorch cross entropic loss needs only class labels, not one-hot
	label = tags.index(tag)
	X_train.append(bag)
	y_train.append(label)
X_train = np.array(X_train)
y_train = np.array(y_train)


EPOCHS = 5000
batch_size = 8
lr = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
class chatDataset(Dataset):
	def __init__(self):
		self.n_samples = len(X_train)
		self.x_data = X_train
		self.y_data = y_train
	def __getitem__(self, index):
		return self.x_data[index], self.y_data[index]
	def __len__(self):
		return self.n_samples
dataset = chatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size,shuffle=True, num_workers=0)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)




In [20]:
device

device(type='mps')

In [21]:

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Train the model
for epoch in range(EPOCHS):
	for (words,labels) in train_loader:
		words = words.to(device)
		labels = labels.to(device)

		#Forward
		outputs = model(words)
		loss = criterion(outputs, labels)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
	if (epoch+1) % 10 == 0:
		print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}")
print(f"final loss: {loss.item():.4f}")
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [10/1000], Loss: 2.2127
Epoch [20/1000], Loss: 1.0162
Epoch [30/1000], Loss: 1.6551
Epoch [40/1000], Loss: 1.3703
Epoch [50/1000], Loss: 1.4050
Epoch [60/1000], Loss: 0.0123
Epoch [70/1000], Loss: 0.9023
Epoch [80/1000], Loss: 0.5893
Epoch [90/1000], Loss: 0.1970
Epoch [100/1000], Loss: 0.0962
Epoch [110/1000], Loss: 0.0060
Epoch [120/1000], Loss: 0.0472
Epoch [130/1000], Loss: 0.0062
Epoch [140/1000], Loss: 0.4840
Epoch [150/1000], Loss: 0.0708
Epoch [160/1000], Loss: 0.1445
Epoch [170/1000], Loss: 0.0029
Epoch [180/1000], Loss: 0.0254
Epoch [190/1000], Loss: 0.0126
Epoch [200/1000], Loss: 0.0004
Epoch [210/1000], Loss: 0.2283
Epoch [220/1000], Loss: 0.0034
Epoch [230/1000], Loss: 0.0062
Epoch [240/1000], Loss: 0.0019
Epoch [250/1000], Loss: 0.0015
Epoch [260/1000], Loss: 0.2436
Epoch [270/1000], Loss: 0.2356
Epoch [280/1000], Loss: 0.0021
Epoch [290/1000], Loss: 0.0037
Epoch [300/1000], Loss: 0.0016
Epoch [310/1000], Loss: 0.0014
Epoch [320/1000], Loss: 0.0004
Epoch [330/1000],